In [3]:
!pip install gensim

In [4]:
# Import libraries to build Word2Vec model, and load Newsgroups data
import os
import numpy as np
import sys
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
import pandas as pd

In [11]:
#Import data
df1 = pd.read_csv("True.csv")
data1 = df1["text"]
df2 = pd.read_csv("Fake.csv")
data2 = df2["text"]
data = pd.concat([data1, data2], ignore_index=True)
y = np.hstack((np.zeros(len(data1)), np.ones(len(data2))))

# Remove Special Characters and append to corpus

dumb_files = []
corpus = []
for i, text in enumerate(data):
    try:
        text_mod = re.sub("[^a-zA-Z]", " ", text)
        text_mod = text_mod.lower()
        text_mod = text_mod.split()

        filt = [w for w in text_mod if len(w) > 1]
        if len(filt) > 3:
            corpus.append(filt)
        else:
            dumb_files.append(i)
        
    except TypeError:
        dumb_files.append(i)

print('Der er ',len(data),' artikler i datafilen')
print('Der er ',len(corpus),' artikler i corpus')
print('Forskellen er', len(data)-len(corpus), 'artikler' )

y = np.delete(y, dumb_files)


Der er  44898  artikler i datafilen
Der er  44218  artikler i corpus
Forskellen er 680 artikler


In [14]:
for i in corpus:
    if len(i) < 3:
        print(i)

In [13]:
#train model
model = Word2Vec(corpus, min_count=0)

#summarize the loaded model
print(model)

#summarize vocabulary into list
words = list(model.wv.index_to_key)
# print(words)

#access vector for one word
# print(model.wv['trump'])
# print(model.wv.most_similar('fish', topn=10))

#save model
model.save('model.selftrained')

#load model
new_model = Word2Vec.load('model.selftrained')
print(new_model)

Word2Vec(vocab=115465, vector_size=100, alpha=0.025)
Word2Vec(vocab=115465, vector_size=100, alpha=0.025)


In [15]:
word_to_test = 'it'
print(model.wv.most_similar(word_to_test,topn=10))


[('he', 0.7037044763565063), ('that', 0.6904121041297913), ('this', 0.6482501029968262), ('something', 0.6327779293060303), ('but', 0.6250712871551514), ('she', 0.6169869303703308), ('really', 0.6001038551330566), ('simply', 0.5811920762062073), ('anything', 0.5782747268676758), ('what', 0.5712040066719055)]


In [16]:
vector_length = 100
art_matrix = np.zeros((len(corpus), vector_length))

for i, text in enumerate(corpus):
    vect = [model.wv[w] for w in text]
    x = np.mean(vect, axis=0)
    art_matrix[i] = x

# print(art_matrix)

In [17]:
x = np.argwhere(np.isnan(art_matrix))
print(x)
row = 0
for i in x[:, 0]:
    if i > row:
        # print(corpus[i])
        row = i

[]


In [20]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000)
from sklearn.model_selection import cross_val_score
first_scores = cross_val_score(log_reg, art_matrix, y, cv=5)
print("Average cross-validation score: {:.2f}".format(first_scores.mean()))

Average cross-validation score: 0.95
